In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model, decomposition, manifold, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import precision_recall_curve, roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import wasserstein_distance
import pickle
import time
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error
from utils.utils import get_max_value_image_coordinates

import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))
os.environ["CUDA_VISIBLE_DEVICES"]="1"

sns.set_theme()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Proton data

In [3]:
data = pd.read_pickle('../../data/data_proton_photonsum_proton_1_2312.pkl')

In [4]:
positions = []
for img in data:
    coordinates_max = get_max_value_image_coordinates(img)
    positions.append(coordinates_max)

In [5]:
coordinates_df = pd.DataFrame(data=positions, columns=('max_x', 'max_y'))

In [6]:
coordinates_df.head()

,max_x,max_y
0,11,20
1,37,2
2,12,13
3,17,0
4,12,28


In [7]:
coordinates_df.to_pickle('../../data/data_coord_proton_photonsum_proton_1_2312.pkl')

# Padded version of dataset

In [5]:
def get_max_value_image_coordinates_padded(img):
    """
    :param img: Input image of any shape
    :return: Tuple with (X, Y) coordinates
    """
    data_proton = img[:, :, 0]
    data_neutron = img[:, :, 1]
    return np.unravel_index(np.argmax(data_proton), data_proton.shape), np.unravel_index(np.argmax(data_neutron), data_neutron.shape)

In [6]:
data = pd.read_pickle('../../data/data_proton_neutron_photonsum_proton_18_1970_neutron_18_3249_padding.pkl')
print('Loaded: ',  data.shape, "max:", data.max())

Loaded:  (38348, 56, 44, 2) max: 603.0


In [7]:
positions_p = []
positions_n = []
for img in data:
    coordinates_proton, coordinates_proton = get_max_value_image_coordinates_padded(img)
    positions_p.append(coordinates_proton)
    positions_n.append(coordinates_proton)

In [8]:
concatenation_coordinates = np.concatenate([positions_p, positions_n], axis=1)

In [9]:
coordinates_df = pd.DataFrame(data=concatenation_coordinates, columns=('max_x_proton', 'max_y_proton', 'max_x_neutron', 'max_y_neutron'))
coordinates_df

,max_x_proton,max_y_proton,max_x_neutron,max_y_neutron
0,43,16,43,16
1,35,32,35,32
2,48,25,48,25
3,46,27,46,27
4,44,27,44,27
...,...,...,...,...
38343,41,26,41,26
38344,27,27,27,27
38345,49,19,49,19
38346,19,42,19,42


In [10]:
coordinates_df.to_pickle('../../data/data_coord_proton_neutron_photonsum_18.pkl')